In [1]:
from_symbol = 'BTC'
to_symbol = 'USD'
exchange = 'Bitstamp'
datetime_interval = 'day'

In [3]:
import requests
import pandas as pd
from datetime import datetime


def get_filename(from_symbol, to_symbol, exchange, datetime_interval, download_date):
    return '%s_%s_%s_%s_%s.csv' % (from_symbol, to_symbol, exchange, datetime_interval, download_date)


def download_data(from_symbol, to_symbol, exchange, datetime_interval):
    supported_intervals = {'minute', 'hour', 'day'}
    assert datetime_interval in supported_intervals,\
        'datetime_interval should be one of %s' % supported_intervals

    print('Downloading %s trading data for %s %s from %s' %
          (datetime_interval, from_symbol, to_symbol, exchange))
    base_url = 'https://min-api.cryptocompare.com/data/histo'
    url = '%s%s' % (base_url, datetime_interval)

    params = {'fsym': from_symbol, 'tsym': to_symbol,
              'limit': 2000, 'aggregate': 1,
              'e': exchange}
    request = requests.get(url, params=params)
    data = request.json()
    return data


def convert_to_dataframe(data):
    df = pd.io.json.json_normalize(data, ['Data'])
    df['datetime'] = pd.to_datetime(df.time, unit='s')
    df = df[['datetime', 'low', 'high', 'open',
             'close', 'volumefrom', 'volumeto']]
    return df


def filter_empty_datapoints(df):
    indices = df[df.sum(axis=1) == 0].index
    print('Filtering %d empty datapoints' % indices.shape[0])
    df = df.drop(indices)
    return df


data = download_data(from_symbol, to_symbol, exchange, datetime_interval)
df = convert_to_dataframe(data)
df = filter_empty_datapoints(df)

current_datetime = datetime.now().date().isoformat()
filename = get_filename(from_symbol, to_symbol, exchange, datetime_interval, current_datetime)
print('Saving data to %s' % filename)
df.to_csv(filename, index=False)

Filtering 595 empty datapoints
Saving data to BTC_USD_Bitstamp_day_2018-10-03.csv


In [4]:
import pandas as pd

def read_dataset(filename):
    print('Reading data from %s' % filename)
    df = pd.read_csv(filename)
    df.datetime = pd.to_datetime(df.datetime) # change type from object to datetime
    df = df.set_index('datetime') 
    df = df.sort_index() # sort by datetime
    print(df.shape)
    return df

df = read_dataset(filename)

Reading data from BTC_USD_Bitstamp_day_2018-10-03.csv
(1406, 6)


In [5]:
df.head()

low    high    open   close  volumefrom    volumeto
datetime                                                          
2014-11-28  360.57  381.34  363.59  376.28     8617.15  3220878.18
2014-11-29  372.25  386.60  376.42  376.72     7245.19  2746157.05
2014-11-30  373.32  381.99  376.57  373.34     3046.33  1145566.61
2014-12-01  373.03  382.31  376.40  378.39     6660.56  2520662.37
2014-12-02  375.23  382.86  378.39  379.25     6832.53  2593576.46

In [7]:
from stockstats import StockDataFrame
df = StockDataFrame.retype(df)
df['macd'] = df.get('macd') # calculate MACD

In [8]:
df.head()

low    high    open   close  volumefrom    volumeto  \
datetime                                                             
2014-11-28  360.57  381.34  363.59  376.28     8617.15  3220878.18   
2014-11-29  372.25  386.60  376.42  376.72     7245.19  2746157.05   
2014-11-30  373.32  381.99  376.57  373.34     3046.33  1145566.61   
2014-12-01  373.03  382.31  376.40  378.39     6660.56  2520662.37   
2014-12-02  375.23  382.86  378.39  379.25     6832.53  2593576.46   

            close_12_ema  close_26_ema      macd     macds     macdh  
datetime                                                              
2014-11-28    376.280000    376.280000  0.000000  0.000000  0.000000  
2014-11-29    376.518333    376.508462  0.009872  0.005484  0.008775  
2014-11-30    375.277829    375.370064 -0.092235 -0.034565 -0.115341  
2014-12-01    376.260220    376.214306  0.045914 -0.007302  0.106432  
2014-12-02    377.072532    376.918296  0.154236  0.040752  0.226969

In [9]:
from math import pi

from bokeh.plotting import figure, show, output_notebook, output_file
output_notebook()

datetime_from = '2016-01-01 00:00'
datetime_to = '2017-12-10 00:00'


def get_candlestick_width(datetime_interval):
    if datetime_interval == 'minute':
        return 30 * 60 * 1000  # half minute in ms
    elif datetime_interval == 'hour':
        return 0.5 * 60 * 60 * 1000  # half hour in ms
    elif datetime_interval == 'day':
        return 12 * 60 * 60 * 1000  # half day in ms


df_limit = df[datetime_from: datetime_to].copy()
inc = df_limit.close > df_limit.open
dec = df_limit.open > df_limit.close

title = '%s datapoints from %s to %s for %s and %s from %s with MACD strategy' % (
    datetime_interval, datetime_from, datetime_to, from_symbol, to_symbol, exchange)
p = figure(x_axis_type="datetime",  plot_width=1000, title=title)

p.line(df_limit.index, df_limit.close, color='black')

# plot macd strategy
p.line(df_limit.index, 0, color='black')
p.line(df_limit.index, df_limit.macd, color='blue')
p.line(df_limit.index, df_limit.macds, color='orange')
p.vbar(x=df_limit.index, bottom=[
       0 for _ in df_limit.index], top=df_limit.macdh, width=4, color="purple")

# plot candlesticks
candlestick_width = get_candlestick_width(datetime_interval)
p.segment(df_limit.index, df_limit.high,
          df_limit.index, df_limit.low, color="black")
p.vbar(df_limit.index[inc], candlestick_width, df_limit.open[inc],
       df_limit.close[inc], fill_color="#D5E1DD", line_color="black")
p.vbar(df_limit.index[dec], candlestick_width, df_limit.open[dec],
       df_limit.close[dec], fill_color="#F2583E", line_color="black")

output_file("visualizing_trading_strategy.html", title="visualizing trading strategy")
show(p)

Loading BokehJS ...

In [13]:
import pandas as pd

def read_dataset(filename):
    print('Reading data from %s' % filename)
    df = pd.read_csv(filename)
    df.datetime = pd.to_datetime(df.datetime) # change type from object to datetime
    df = df.set_index('datetime') 
    df = df.sort_index() # sort by datetime
    print(df.shape)
    return df

filename = '/Users/zhaocai/Downloads/courses/MSBD5013_Statistical_Prediction/data/BTC-USD_T.csv'
df = pd.read_csv(filename)

In [14]:
df.head()

time  close   high    low   open  volume
0  2014-12-01 05:33:00  300.0  300.0  300.0  300.0    0.01
1  2014-12-01 05:34:00  300.0  300.0  300.0  300.0    0.00
2  2014-12-01 05:35:00  300.0  300.0  300.0  300.0    0.00
3  2014-12-01 05:36:00  300.0  300.0  300.0  300.0    0.00
4  2014-12-01 05:37:00  300.0  300.0  300.0  300.0    0.00

In [18]:
from stockstats import StockDataFrame
df = StockDataFrame.retype(df)
df['macd'] = df.get('macd') # calculate MACD

In [22]:
df.tail()

time    close     high      low     open    volume  \
1984525  2018-09-09 08:58:00  6192.78  6192.78  6192.77  6192.78  0.735574   
1984526  2018-09-09 08:59:00  6192.78  6192.78  6192.77  6192.77  0.221675   
1984527  2018-09-09 09:00:00  6192.78  6192.78  6192.77  6192.78  0.852274   
1984528  2018-09-09 09:01:00  6192.77  6192.78  6192.77  6192.77  1.563542   
1984529  2018-09-09 09:02:00  6192.77  6192.78  6192.77  6192.78  0.589552   

         close_12_ema  close_26_ema      macd     macds     macdh  
1984525   6191.521610   6191.310331  0.211278 -0.058226  0.539008  
1984526   6191.715208   6191.419196  0.296012  0.012622  0.566781  
1984527   6191.879022   6191.519996  0.359026  0.081903  0.554247  
1984528   6192.016096   6191.612589  0.403507  0.146224  0.514567  
1984529   6192.132081   6191.698323  0.433758  0.203731  0.460055

In [21]:
print(df['macd'])

0          0.000000
1          0.000000
2          0.000000
3          0.000000
4          0.000000
5          0.000000
6          0.000000
7          0.000000
8          0.000000
9          0.000000
10         0.000000
11         0.000000
12         0.000000
13         0.000000
14         0.000000
15         0.000000
16         0.000000
17         0.000000
18         0.000000
19         0.000000
20         0.000000
21         0.000000
22         0.000000
23         0.000000
24         0.000000
25         0.000000
26         0.000000
27         0.000000
28         0.000000
29         0.000000
             ...   
1984500   -0.360260
1984501   -0.333491
1984502   -0.308717
1984503   -0.285789
1984504   -0.132147
1984505   -0.142688
1984506   -0.148523
1984507   -0.152199
1984508   -0.153345
1984509   -0.151698
1984510   -0.149476
1984511   -0.145234
1984512   -0.140256
1984513   -0.133959
1984514   -0.128297
1984515   -0.121601
1984516   -0.115767
1984517   -0.109079
1984518   -0.103394


In [23]:
from math import pi

from bokeh.plotting import figure, show, output_notebook, output_file
output_notebook()

datetime_from = '2016-01-01 00:00'
datetime_to = '2017-12-10 00:00'


def get_candlestick_width(datetime_interval):
    if datetime_interval == 'minute':
        return 30 * 60 * 1000  # half minute in ms
    elif datetime_interval == 'hour':
        return 0.5 * 60 * 60 * 1000  # half hour in ms
    elif datetime_interval == 'day':
        return 12 * 60 * 60 * 1000  # half day in ms


df_limit = df[datetime_from: datetime_to].copy()
inc = df_limit.close > df_limit.open
dec = df_limit.open > df_limit.close

title = '%s datapoints from %s to %s for %s and %s from %s with MACD strategy' % (
    datetime_interval, datetime_from, datetime_to, from_symbol, to_symbol, exchange)
p = figure(x_axis_type="datetime",  plot_width=1000, title=title)

p.line(df_limit.index, df_limit.close, color='black')

# plot macd strategy
p.line(df_limit.index, 0, color='black')
p.line(df_limit.index, df_limit.macd, color='blue')
p.line(df_limit.index, df_limit.macds, color='orange')
p.vbar(x=df_limit.index, bottom=[
       0 for _ in df_limit.index], top=df_limit.macdh, width=4, color="purple")

# plot candlesticks
candlestick_width = get_candlestick_width(datetime_interval)
p.segment(df_limit.index, df_limit.high,
          df_limit.index, df_limit.low, color="black")
p.vbar(df_limit.index[inc], candlestick_width, df_limit.open[inc],
       df_limit.close[inc], fill_color="#D5E1DD", line_color="black")
p.vbar(df_limit.index[dec], candlestick_width, df_limit.open[dec],
       df_limit.close[dec], fill_color="#F2583E", line_color="black")

output_file("visualizing_trading_strategy.html", title="visualizing trading strategy")
show(p)


Loading BokehJS ...

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [2016-01-01 00:00] of <class 'str'>

In [ ]:
import pandas as pd
import numpy as np
import datetime
import time
#获取数据
filename = '/Users/zhaocai/Downloads/courses/MSBD5013_Statistical_Prediction/data/BTC-USD_T.csv'
df=pd.read_csv(filename)
'''
df.columns=['date','code','name','close','high','low','open','preclose',
'change','change_per','volume','amt']
df=df[['date','open','high','low','close','volume','amt']]
'''
df.head()
def get_EMA(df,N):
    for i in range(len(df)):
        if i==0:
            df.ix[i,'ema']=df.ix[i,'close']
        if i>0:
            df.ix[i,'ema']=(2*df.ix[i,'close']+(N-1)*df.ix[i-1,'ema'])/(N+1)
    ema=list(df['ema'])
    return ema
def get_MACD(df,short=12,long=26,M=9):
    a=get_EMA(df,short)
    b=get_EMA(df,long)
    df['diff']=pd.Series(a)-pd.Series(b)
    #print(df['diff'])
    for i in range(len(df)):
        if i==0:
            df.ix[i,'dea']=df.ix[i,'diff']
        if i>0:
            df.ix[i,'dea']=(2*df.ix[i,'diff']+(M-1)*df.ix[i-1,'dea'])/(M+1)
    df['macd']=2*(df['diff']-df['dea'])
    return df
get_MACD(df,12,26,9)
df


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [ ]:
df.head()